In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [4]:
# def maskS2clouds(image):
#   qa = image.select('QA60');

#   # Bits 10 and 11 are clouds and cirrus, respectively.
#   cloudBitMask = 1 << 10
#   cirrusBitMask = 1 << 11

#   # Both flags should be set to zero, indicating clear conditions.
#   mask = qa.bitwiseAnd(cloudBitMask).eq(0).and(qa.bitwiseAnd(cirrusBitMask).eq(0)
#   return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start']);

SyntaxError: invalid syntax (2537947249.py, line 10)

In [7]:
def get_s2_sr_cld_col(aoi, start_year, end_year, start_month, end_month):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(start_year,end_year,'year'))
        .filter(ee.Filter.calendarRange(start_month,end_month,'month'))

        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
        .filterBounds(aoi.geometry().centroid())
        )

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filter(ee.Filter.calendarRange(start_year,end_year,'year'))
        .filter(ee.Filter.calendarRange(start_month,end_month,'month'))
    )

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [13]:
def clp(img):
    return img.clip(AOI)

In [21]:



def mosaicByDate(imcol):
    def get_unique_dates(im):
        return ee.Image(im).date().format("YYYY-MM-dd")
    def get_mosaic_imlist(d):
        d = ee.Date(d)
        im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
        return im.set(
            "system:time_start", d.millis(), 
            "system:id", d.format("YYYY-MM-dd"))

    # imcol: An image collection
    # returns: An image collection
    imlist = imcol.toList(imcol.size())
    unqiue_dates = imlist.map(get_unique_dates).distinct()
    mosaic_imlist = unqiue_dates.map(get_mosaic_imlist)
    return ee.ImageCollection(mosaic_imlist)

In [12]:
HYBAS_ID=3100133910

AOI = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))

centroid = AOI.geometry().centroid()
geometry = centroid.buffer(**{'distance': 20000})

In [15]:
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, 2019, 2019, 5, 8).map(clp)

In [22]:
dataset = mosaicByDate(s2_sr_cld_col_eval)

In [26]:
# Import the folium library.
import folium

# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer